### Use newsapi to collect all pieces of news about trade war and store them in jsons

In [1]:
import requests

In [2]:
import pprint

In [80]:
def generate_url(page):
    url = ('https://newsapi.org/v2/everything?'
       'q="trade war"&'
       'sources=["the-new-york-times","cnn","xinhua-net"]&'
       'from=2018-10-27&'
       'to=2018-11-07&'
       'sortBy=relevancy&'
       'pageSize=100&'
       f'page={page}&'
       'apiKey=fc8921bcf53e482ba461714a254c0d7c'
       )
    return url

In [81]:
result = []
for i in range(1,11):
    response = requests.get(generate_url(i))
    result.extend(response.json()['articles'])       

In [82]:
len(result)

1000

In [19]:
pp = pprint.PrettyPrinter(indent=4)

In [7]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth',10000)

#### To insert data into a DataFrame, it is faster and easier to transform all data into a list of dictionaries and then tranform it into a DataFrame

In [83]:
col = ['title','description','content','url','publishedAt']
df_dict = []
for article in result:
    temp = {}
    for c in col:
        try: 
            temp[c] = article[c]
        except:
            temp[c] = np.nan
    df_dict.append(temp)
df = pd.DataFrame(df_dict,columns=col)
df.to_csv('data/article_info(1027-1107).csv')

In [84]:
df = pd.read_csv('data/article_info.csv')

### Now, try some NLP stuff

In [10]:
import nltk
from nltk.util import ngrams

from textblob import TextBlob

from collections import Counter
from operator import itemgetter

from nltk.corpus import stopwords

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [85]:
# fill null values with empty strings
df.fillna('',inplace=True)

In [86]:
import re
import string

In [87]:
clean_title = []
for title in df.title:
    # Replace punctuations with a white space
    ct = re.sub('[%s]' % re.escape(string.punctuation), ' ', title)
    # Remove all words containing digits
    ct = re.sub('\w*\d\w*', ' ',ct)
    clean_title.append(ct)
print(clean_title[:5])

['Struggling toy industry braces for next round in US China trade war', 'China tariffs are taking a toll on robot vacuums', 'China slashes banks  reserve requirements as trade war imperils growth', 'Trade war  spy claims cloud horizon for China airshow', 'UPS profit gain overshadowed by trade war worries  shares fall']


In [88]:
cv = CountVectorizer(stop_words='english',ngram_range=(1,3),min_df=2)

cv.fit(clean_title)

print(cv.get_feature_names())

['aapl', 'abandon', 'abandons', 'abe', 'activity', 'activity slows', 'activity slows october', 'add', 'add dour', 'add dour mood', 'adding', 'adding fuel', 'adding fuel trade', 'administration', 'administration plans', 'administration plans help', 'advice', 'adviser', 'affairs', 'ahead', 'ahead meeting', 'ahead november', 'ahead november elections', 'ai', 'aim', 'aims', 'air', 'airbus', 'airbus china', 'airshow', 'alibaba', 'alibaba jack', 'alibaba jack ma', 'allies', 'aluminum', 'amazon', 'ambassador', 'ambassador says', 'ambassador says beijing', 'amd', 'america', 'america awaits', 'america awaits midterm', 'american', 'american manufacturers', 'american manufacturers seriously', 'americans', 'amid', 'amid global', 'amid influence', 'amid influence battle', 'amid report', 'amid report trump', 'amid trade', 'amid trade war', 'amzn', 'analyst', 'analysts', 'analysts say', 'antonio', 'anxiety', 'anxiety wisconsin', 'anxiety wisconsin manufacturers', 'apparel', 'apparel industry', 'apple

In [97]:
from collections import defaultdict

In [111]:
# Top bigrams in titles
stop = stopwords.words('english')
stop += ['.', ',', '(', ')', "'", '"']
stop = set(stop)

counter = Counter()

n = 2
for title in df.title:
    words = TextBlob(title).words
    words = [w.lower() for w in words if w not in stop]
    bigrams = ngrams(words, n)
    counter += Counter(bigrams)

for phrase, count in counter.most_common(30):
    print('%20s %i' % (" ".join(phrase), count))

           trade war 353
            trump 's 54
            china 's 49
      us-china trade 40
            's trade 34
         wall street 27
             china ’ 23
          amid trade 23
             trump ’ 21
          yahoo news 21
    u.s.-china trade 18
        donald trump 18
           war china 17
             ’ trade 17
         china trade 16
              war is 14
            us trade 13
     washington post 13
          trade deal 12
        stock market 12
           the trade 12
     economic growth 11
             wall st 10
         trump trade 10
            us china 10
          oil prices 10
       global growth 9
           war could 9
             here 's 9
      trade tensions 9


In [114]:
# Top bigrams in descriptions
stop = stopwords.words('english')
stop += ['.', ',', '(', ')', "'", '"']
stop = set(stop)

counter = Counter()

n = 2
for des in df.description:
    words = TextBlob(des).words
    words = [w.lower() for w in words if w not in stop]
    bigrams = ngrams(words, n)
    counter += Counter(bigrams)

for phrase, count in counter.most_common(30):
    print('%20s %i' % (" ".join(phrase), count))

           trade war 384
       united states 114
        donald trump 103
            china 's 99
    president donald 68
         wall street 49
            trump 's 45
      us-china trade 44
           war china 41
          yahoo news 39
          xi jinping 37
     economic growth 33
             trump ’ 31
          war united 31
    escalating trade 30
             china ’ 30
     washington post 29
trump administration 28
      trade tensions 27
            's trade 27
    u.s.-china trade 26
      interest rates 26
            world 's 25
            new york 24
              war us 23
international monetary 22
       monetary fund 22
     president trump 22
        president xi 21
       stock markets 19


In [92]:
import matplotlib.pyplot as plt

%matplotlib inline

In [109]:
plt.bar(phrase_count.keys(),phrase_count.values())

TypeError: unhashable type: 'slice'

In [110]:
words_corpus = set(words.words())
analyzer = CountVectorizer().build_analyzer()

def english_corpus(doc):
    return [w for w in analyzer(doc) if w in words_corpus]

cv = CountVectorizer(stop_words='english', analyzer=english_corpus, min_df = 2, max_df = .95, ngram_range=(1, 2),
                     strip_accents='unicode', encoding='utf-8')

AttributeError: 'list' object has no attribute 'words'